# Code for generating SLFs for NSD

Story Loss Function Generator for Non-Structural Drift Sensitive Components (SLF_NSD)
=====================================================================================

Author: Shiva Baddipalli
Institution: Utah State University, USA
Email: shivalinga.baddipalli@usu.edu
Last Updated: June 25, 2025

This script generates Story Loss Functions for non-structural drift sensitive
components that respond to inter-story drift demands.

Reference:
----------
Shahnazaryan, D., O'Reilly, G.J., and Monteiro, R. (2021). "Story loss functions 
for seismic design and assessment: Development of tools and application." 
Earthquake Spectra, 37(4), 2813-2839.

SLF Generation Tool: storeyloss Python package
GitHub: https://github.com/davitshahnazaryan3/SLFGenerator

INPUTS REQUIRED:
---------------
1. CSV Files (Component Inventory):
   - File format: "My_Inventory_CASE{case}.csv" where case = 1,2,3
   - Location: "MyInventory_NSD/"
   - Required columns: Component ID, EDP, Component Group, Quantity, Damage States,
     DS1-DS5 (Median Demand), Fragility parameters, Repair costs

2. Components Analyzed:
   - Non-structural drift sensitive components (B1-B4)
   - Components sensitive to inter-story drift ratio (IDR)

3. Case Variations: 
   - 3 cases with different bay width configurations
   - Alternative: 15 cases available (commented out in code)

4. Analysis Parameters:
   - EDP Range: 0-300% inter-story drift ratio, 0.001 increment
   - Correlation Model: 'psd' (peak inter-story drift)
   - Monte Carlo Realizations: 20,000

OUTPUTS GENERATED:
-----------------
- JSON files: output_{case}.json (individual case loss functions)
- CSV file: CombinedOutput_SLF_NSD.csv (consolidated all cases)

HOW STOREYLOSS LIBRARY IS USED:
-------------------------------
1. storeyloss.SLF() - Initializes the SLF model for NSD components:
   - component_data: DataFrame with NSD component inventory (from CSV)
   - 'psd': Correlation model for non-structural component dependencies
   - edp_range: Array of inter-story drift ratio values (0-300%)
   - do_grouping=True: Groups similar NSD components for efficiency
   - realizations=20000: Monte Carlo simulations for uncertainty quantification

2. model.generate_slfs() - Executes NSD component SLF generation:
   - Performs Monte Carlo simulations across drift range
   - Calculates NSD damage probabilities using drift-based fragility functions
   - Estimates repair/replacement costs for damaged NSD components
   - Aggregates NSD losses to story-level functions

3. model.export_to_json() - Exports NSD loss function results:
   - Saves loss functions as drift ratio vs. loss relationships
   

In [ ]:
import pandas as pd
import json
import storeyloss
import numpy as np

# Configuration
cases = [1, 2, 3]
#cases = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
base_path = "MyInventory_NSD/"

# Initialize DataFrame for combined results
combined_df = pd.DataFrame()

# Process each case
for case in cases:
    print(f"Processing NSD Case {case}...")
    
    # Load component inventory data
    component_data = pd.read_csv(f"{base_path}My_Inventory_CASE{case}.csv")
    
    # Define EDP range (0% to 300% inter-story drift ratio)
    edp_range = np.arange(0, 3.001, 0.001)
    
    # Initialize SLF model
    model = storeyloss.SLF(
        component_data,
        'psd',                    
        edp_range=edp_range,
        do_grouping=True,         # Enable component grouping
        realizations=20000        # Monte Carlo simulations
    )
    
    # Generate story loss functions
    out = model.generate_slfs()
    
    # Export to JSON
    output_json_path = f"{base_path}output_{case}.json"
    model.export_to_json(out, output_json_path)
    
    # Load JSON data for combination
    with open(output_json_path, 'r') as file:
        data = json.load(file)
    
    # Extract EDP range and SLF data from group '2' (NSD components)
    edp_range = data['2']['edp_range']
    slf = data['2']['slf']
    
    # Set EDP range column (only for first case)
    if case == 1:
        combined_df['EDP Range'] = edp_range
    
    # Add SLF data for current case
    combined_df[f'SLF Case {case}'] = slf
    
    print(f"NSD Case {case} completed")

# Save combined results
output_csv = f'{base_path}CombinedOutput_SLF_NSD.csv'
combined_df.to_csv(output_csv, index=False)

print(f"\n{'='*60}")
print("NON-STRUCTURAL DRIFT SENSITIVE SLF GENERATION COMPLETED")
print(f"{'='*60}")
print(f"Processed {len(cases)} NSD cases")
print(f"Combined results saved to: {output_csv}")

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)
